In [1]:
import FuzzyMatch as fm
import ConfigReader as cr
import ComposeFunction as cf
import pandas as pd
import numpy as np

path = "tests/"


# Load data
def load_match_data():
    companies = pd.read_csv(path + "data/fuzzy_match_companies.csv")
    return companies

In [2]:
# Load data
companies = load_match_data()
config = cr.read_config("config/cleaning/clean_company_name.ini")
fns = cr.parse_functions(config)
ref_name = cf.eval_functions_list(companies["reference_name"], fns)
new_name = cf.eval_functions_list(companies["new_name"], fns)

In [4]:
ref_name

['ballentine partners',
 'huron capital partners',
 'usi insurance svc',
 'new heritage capital',
 'aureus',
 'marsh usa',
 'madison capital',
 'adams street partners',
 'promus bank trust',
 'huron consulting group',
 'comprehensive capital partners',
 'andrews banking',
 'piper sandler',
 'peabody partners',
 'the mortgage office',
 'john deere']

In [7]:
cf.eval_functions_list("Acme Corp £", fns)

'acme'

In [37]:
companies["result"] = np.where(companies["scores"] >= 80, "match", "new")
companies["expected"].to_list() == companies["result"].to_list()

True

In [38]:
print(companies)

                    reference_name                        new_name expected  scores result
0          Ballentine Partners LLC        Ballentine Partners, LLC    match     100  match
1       Huron Capital Partners LLC                   Huron Capital    match      90  match
2                USI Insurance Svc          Usi Insurance Services    match      87  match
3        New Heritage Capital, LLC            New Heritage Capital    match     100  match
4                           Aureus                  Aureus Capital    match      90  match
5                  Marsh USA, Inc.                       Marsh Inc    match      90  match
6              Madison Capital LLC         Madison Capital Funding    match      90  match
7        Adams Street Partners LLC              Partners, Adams St    match      86  match
8              Promus Bank & Trust              Promus Capital LLC      new      58    new
9           Huron Consulting Group      Huron Capital Partners LLC      new      50    new